<a href="https://colab.research.google.com/github/kumar-abhishek/handson-ml2/blob/master/Normalized_BachChorales_HandsOnChapter_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# univariate lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
     break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
print(X)
print(y)

[[10 20 30]
 [20 30 40]
 [30 40 50]
 [40 50 60]
 [50 60 70]
 [60 70 80]]
[40 50 60 70 80 90]


In [0]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features))) model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
                                                                                   
print(yhat)

In [0]:
# multivariate output stacked lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
  X, y = list(), list()
  for i in range(len(sequences)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the dataset
    if end_ix > len(sequences)-1:
     break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
    X.append(seq_x)
    y.append(seq_y)
    return array(X), array(y)
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
print(dataset)
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X)
print(y)
print(type(X))
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
print(n_features)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]
[[[10 15 25]
  [20 25 45]
  [30 35 65]]]
[[40 45 85]]
<class 'numpy.ndarray'>
3


In [0]:
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu')) 
model.add(Dense(n_features)) 
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=1000, verbose=0)


In [0]:
# demonstrate prediction
x_input = array([[70,75,145], [80,85,165], [90,95,185]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[197.93314 223.3317  407.88092]]


**Q10[part-1]. Download the Bach chorales dataset and unzip it. It is composed of 382 chorales composed by Johann Sebastian Bach. Each chorale is 100 to 640 time steps long, and each time step contains 4 integers, where each integer corresponds to a note’s index on a piano (except for the value 0, which means that no note is played). Train a model—recurrent, convolutional, or both—that can predict the next time step (four notes), given a sequence of time steps from a chorale. **bold text**

In [5]:
from google.colab import files
uploaded = files.upload()

Saving chorale_229.csv to chorale_229.csv


In [7]:
uploaded_validation = files.upload()

Saving chorale_000.csv to chorale_000.csv


In [78]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
import numpy as np

X_train_000 = pd.read_csv('chorale_000.csv')
print(X_train_000.head(10))

X_valid_229 = pd.read_csv('chorale_229.csv')
print(X_valid_229.head(10))

X_train_000 = X_train_000.to_numpy()
X_valid_229 = X_valid_229.to_numpy()

dataX=X_train_000[:-1]
dataY=X_train_000[1:]
dataValidationX=X_valid_229[:-1]
dataValidationY=X_valid_229[1:]

print(dataX[0:10])
print(dataY[0:10])
print(type(dataX))

scaler = MinMaxScaler( feature_range=(0, 1) )
dataXScaler = scaler.fit_transform(dataX)
dataYScaler = scaler.fit_transform(dataY)
print(dataXScaler[0:10])
print(dataYScaler[0:10])

dataValidationX = scaler.fit_transform(dataValidationX)
dataValidationY = scaler.fit_transform(dataValidationY)
print(dataValidationX[0:5])
print(dataValidationY[0:5])

   note0  note1  note2  note3
0     74     70     65     58
1     74     70     65     58
2     74     70     65     58
3     74     70     65     58
4     75     70     58     55
5     75     70     58     55
6     75     70     60     55
7     75     70     60     55
8     77     69     62     50
9     77     69     62     50
   note0  note1  note2  note3
0     72     67     60     48
1     72     67     60     48
2     72     67     60     48
3     72     67     60     48
4     72     67     64     48
5     72     67     64     48
6     72     67     64     50
7     72     67     64     50
8     72     67     64     52
9     72     67     64     52
[[74 70 65 58]
 [74 70 65 58]
 [74 70 65 58]
 [74 70 65 58]
 [75 70 58 55]
 [75 70 58 55]
 [75 70 60 55]
 [75 70 60 55]
 [77 69 62 50]
 [77 69 62 50]]
[[74 70 65 58]
 [74 70 65 58]
 [74 70 65 58]
 [75 70 58 55]
 [75 70 58 55]
 [75 70 60 55]
 [75 70 60 55]
 [77 69 62 50]
 [77 69 62 50]
 [77 69 62 50]]
<class 'numpy.ndarray'>
[[0.44444444 0

In [79]:
# reference: https://machinelearningmastery.com/how-to-use-the-timeseriesgenerator-for-time-series-forecasting-in-keras/
from keras.preprocessing.sequence import TimeseriesGenerator
# make length as a hyperparameter and find it's optimal value via GridSearchCV?
generator = TimeseriesGenerator(dataXScaler, dataYScaler, length=3, batch_size=32)
validation_generator = TimeseriesGenerator(dataValidationX, dataValidationY, length=3, batch_size=32)
print(generator[0])

(array([[[0.44444444, 0.77777778, 1.        , 0.88235294],
        [0.44444444, 0.77777778, 1.        , 0.88235294],
        [0.44444444, 0.77777778, 1.        , 0.88235294]],

       [[0.44444444, 0.77777778, 1.        , 0.88235294],
        [0.44444444, 0.77777778, 1.        , 0.88235294],
        [0.44444444, 0.77777778, 1.        , 0.88235294]],

       [[0.44444444, 0.77777778, 1.        , 0.88235294],
        [0.44444444, 0.77777778, 1.        , 0.88235294],
        [0.55555556, 0.77777778, 0.41666667, 0.70588235]],

       [[0.44444444, 0.77777778, 1.        , 0.88235294],
        [0.55555556, 0.77777778, 0.41666667, 0.70588235],
        [0.55555556, 0.77777778, 0.41666667, 0.70588235]],

       [[0.55555556, 0.77777778, 0.41666667, 0.70588235],
        [0.55555556, 0.77777778, 0.41666667, 0.70588235],
        [0.55555556, 0.77777778, 0.58333333, 0.70588235]],

       [[0.55555556, 0.77777778, 0.41666667, 0.70588235],
        [0.55555556, 0.77777778, 0.58333333, 0.70588235],
   

In [80]:
from keras.models import Sequential
from keras import optimizers
from keras.layers import LSTM
from keras.layers import Dense, Dropout, BatchNormalization
# the dataset knows the number of features, e.g. 2
n_features = 4

# choose a number of time steps
n_steps = None

model = Sequential()
model.add(LSTM(64, activation='tanh', input_shape=(None, n_features), return_sequences=True))
model.add(BatchNormalization())

model.add(LSTM(32 , activation = 'tanh'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='categorical_crossentropy')

# fit model
model.fit_generator(generator, epochs=500, validation_data=validation_generator)

Epoch 1/500
6/6 [==============================] - 12s 2s/step - loss: 14.5823 - val_loss: 13.5102
Epoch 2/500
6/6 [==============================] - 0s 18ms/step - loss: 13.1113 - val_loss: 12.9651
Epoch 3/500
6/6 [==============================] - 0s 16ms/step - loss: 12.7951 - val_loss: 13.7556
Epoch 4/500
6/6 [==============================] - 0s 17ms/step - loss: 12.4727 - val_loss: 12.9644
Epoch 5/500
6/6 [==============================] - 0s 17ms/step - loss: 11.5871 - val_loss: 12.6646
Epoch 6/500
6/6 [==============================] - 0s 15ms/step - loss: 11.7360 - val_loss: 12.3133
Epoch 7/500
6/6 [==============================] - 0s 17ms/step - loss: 11.6949 - val_loss: 12.1180
Epoch 8/500
6/6 [==============================] - 0s 16ms/step - loss: 10.7379 - val_loss: 11.5511
Epoch 9/500
6/6 [==============================] - 0s 16ms/step - loss: 11.1687 - val_loss: 11.4793
Epoch 10/500
6/6 [==============================] - 0s 17ms/step - loss: 10.0121 - val_loss: 11.6794


In [77]:
# demonstrate prediction
x_input = dataXScaler[2:15]
x_input = x_input.reshape((1, len(x_input), 4))
#print(x_input)
print(scaler.inverse_transform(dataXScaler)[2:15])
yhat = model.predict(x_input, verbose=0)
#print(yhat)
print(scaler.inverse_transform(yhat))
print('expected: ', scaler.inverse_transform(dataYScaler)[15])

[[74. 70. 65. 58.]
 [74. 70. 65. 58.]
 [75. 70. 58. 55.]
 [75. 70. 58. 55.]
 [75. 70. 60. 55.]
 [75. 70. 60. 55.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 70. 62. 55.]
 [77. 70. 62. 55.]
 [77. 69. 62. 55.]]
[[70.78111  63.217148 58.257935 46.919853]]
expected:  [75. 67. 63. 48.]


**Q10[part-2] Then use this model to generate Bach-like music, one note at a time: you can do this by giving the model the start of a chorale and asking it to predict the next time step, then appending these time steps to the input sequence and asking the model for the next note, and so on. Also make sure to check out Google’s Coconet model, which was used for a nice Google doodle about Bach.**

In [43]:
# Using SimpleRNN 
print(n_features)
model_rnn = keras.models.Sequential([
    keras.layers.SimpleRNN(100, return_sequences=True, input_shape=[None, n_features]),
    keras.layers.SimpleRNN(100),
    keras.layers.Dense(n_features)
])

model_rnn.compile(loss="mse", optimizer="adam")
#print(X.shape, y.shape)
# history = model_rnn.fit(X, y, epochs=500,
#                     validation_data=(validX, validY))

model_rnn.fit_generator(generator, epochs=500, validation_data=validation_generator)

4


AttributeError: ignored

In [0]:
# choose a number of time steps
n_steps = None
# convert into input/output
i=1
n=1
#x_input = X[0]
x_input = np.array([dataX[0]])

x_input = x_input.reshape((1, len(x_input), n_features))
while i<len(X):
  # demonstrate prediction
  #x_input = x_input.reshape((1, len(x_input), n_features))
  print('Input: ', x_input)
  print('---------------')
  print('Expected Output: ', y[i])
  print('---------------')
  yhat = model_rnn.predict(x_input, verbose=1)
  print('Predicted Output: ', yhat)
  print("\n\n")
  yhat = yhat.reshape((1, len(yhat), n_features))
  x_input = np.concatenate([x_input, yhat], axis=1)
  i += 1
  if i>20:
    break

Input:  [[[74 70 65 58]]]
---------------
Expected Output:  [74 70 65 58]
---------------
1/1 [==============================] - 0s 3ms/sample
Predicted Output:  [[72.12551  65.19431  58.163357 50.04461 ]]



Input:  [[[74.         70.         65.         58.        ]
  [72.12551117 65.19431305 58.16335678 50.04460907]]]
---------------
Expected Output:  [74 70 65 58]
---------------
1/1 [==============================] - 0s 2ms/sample
Predicted Output:  [[74.03032  67.032486 59.8005   51.336597]]



Input:  [[[74.         70.         65.         58.        ]
  [72.12551117 65.19431305 58.16335678 50.04460907]
  [74.03031921 67.03248596 59.80049896 51.33659744]]]
---------------
Expected Output:  [75 70 58 55]
---------------
1/1 [==============================] - 0s 2ms/sample
Predicted Output:  [[74.03046  67.032616 59.80062  51.336697]]



Input:  [[[74.         70.         65.         58.        ]
  [72.12551117 65.19431305 58.16335678 50.04460907]
  [74.03031921 67.03248596 59.800

In [0]:

# using GRU
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
print(n_features)

# choose a number of time steps
n_steps = None

model_gru = keras.models.Sequential([
    keras.layers.GRU(100, return_sequences=True, input_shape=[n_steps, n_features]),
    keras.layers.GRU(100),
    keras.layers.Dense(n_features)
])
model_gru.compile(optimizer='adam', loss='mse')
# fit model
model_gru.fit(X, y, epochs=250, validation_data=(validX, validY))


4
Train on 191 samples, validate on 195 samples
Epoch 1/250
191/191 [==============================] - 7s 38ms/sample - loss: 3954.2694 - val_loss: 3808.7841
Epoch 2/250
191/191 [==============================] - 4s 20ms/sample - loss: 3655.6493 - val_loss: 3534.8152
Epoch 3/250
191/191 [==============================] - 4s 20ms/sample - loss: 3411.2531 - val_loss: 3333.9724
Epoch 4/250
191/191 [==============================] - 4s 20ms/sample - loss: 3238.6062 - val_loss: 3211.2050
Epoch 5/250
191/191 [==============================] - 4s 20ms/sample - loss: 3144.1109 - val_loss: 3131.7926
Epoch 6/250
191/191 [==============================] - 4s 21ms/sample - loss: 3067.1131 - val_loss: 3056.0901
Epoch 7/250
191/191 [==============================] - 4s 21ms/sample - loss: 2992.6149 - val_loss: 2982.3321
Epoch 8/250
191/191 [==============================] - 4s 21ms/sample - loss: 2920.1653 - val_loss: 2910.4434
Epoch 9/250
191/191 [==============================] - 4s 20ms/sample - 

KeyboardInterrupt: ignored

In [0]:
# demonstrate prediction
x_input = X[0:5]
x_input = x_input.reshape((1, len(x_input), n_features))
print(x_input)
yhat = model_gru.predict(x_input, verbose=0)
print(yhat)

[[[74 70 65 58]
  [74 70 65 58]
  [74 70 65 58]
  [74 70 65 58]
  [75 70 58 55]]]
[[73.542854 66.80782  59.76219  51.33617 ]]


In [0]:
# choose a number of time steps
n_steps = None
# convert into input/output
i=1
n=1
#x_input = X[0]
x_input = np.array([dataX[0]])
x_input = x_input.reshape((1, len(x_input), n_features))
while i<len(X):
  # demonstrate prediction
  print('Input: ', x_input)
  print('---------------')
  print('Expected Output: ', y[i])
  print('---------------')
  yhat = model_gru.predict(x_input, verbose=1)
  print('Predicated Output: ', yhat)
  print("\n\n")
  yhat = yhat.reshape((1, len(yhat), n_features))
  x_input = np.concatenate([x_input, yhat], axis=1)
  i += 1
  if i>10:
    break

Input:  [[[74 70 65 58]]]
---------------
Expected Output:  [74 70 65 58]
---------------
1/1 [==============================] - 0s 5ms/sample
Predicated Output:  [[73.5445   66.80927  59.76339  51.336994]]



Input:  [[[74.         70.         65.         58.        ]
  [73.54450226 66.80927277 59.76338959 51.33699417]]]
---------------
Expected Output:  [74 70 65 58]
---------------
1/1 [==============================] - 0s 2ms/sample
Predicated Output:  [[73.54287  66.80782  59.762207 51.33618 ]]



Input:  [[[74.         70.         65.         58.        ]
  [73.54450226 66.80927277 59.76338959 51.33699417]
  [73.54286957 66.80782318 59.76220703 51.33618164]]]
---------------
Expected Output:  [75 70 58 55]
---------------
1/1 [==============================] - 0s 2ms/sample
Predicated Output:  [[73.542854 66.807816 59.76219  51.336174]]



Input:  [[[74.         70.         65.         58.        ]
  [73.54450226 66.80927277 59.76338959 51.33699417]
  [73.54286957 66.80782318 59.